In [34]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

import os
import matplotlib.pyplot as plt
import shutil

# Folder structure

# data/train/RETARGETTED    - RETARGETTED train samples
# data/train/NATURAL        - naturaltrain samples
# data/train/DIBR           - DIBR train samples
# data/train/SCREENSHOTS    - SCREENSHOT train samples


# data/test/RETARGETTED    - RETARGETTED test samples
# data/test/NATURAL        - natural test samples
# data/test/DIBR           - DIBR test samples
# data/test/SCREENSHOTS    - SCREENSHOT test samples


In [35]:
img_width, img_height = 150, 150 # Resolution of inputs
train_data_dir = "train"           # Folder of train samples
validation_data_dir = "test" # Folder of validation samples
nb_train_samples =  23774              # Number of train samples
nb_validation_samples =  9773           # Number of validation samples
batch_size = 16                        # Batch size
epochs = 100                # Maximum number of epochs
dataset_path = os.getcwd() + '/dataset' # Place where dataset images are stored
split_percent = 0.7

# Split dataset into train and test

In [ ]:
dataset_path

number_of_classes = len(os.listdir(dataset_path))
classes = list(os.listdir(dataset_path))

os.makedirs(train_data_dir, exist_ok=True)
os.makedirs(validation_data_dir, exist_ok=True)
# every class has 350 samples
for c in classes:
    sample_dir = os.listdir(dataset_path + '/' + c)
    number_sample = len(sample_dir)
    
    print('Class' + c + ' has ' + str(number_sample) + ' samples.')
    
    # create directories to move data
    os.makedirs(train_data_dir + '/' + c, exist_ok = True)
    os.makedirs(validation_data_dir + '/' + c, exist_ok = True)
    
    # move data based on split percentage to TEST
    [shutil.copy(os.path.join(dataset_path, c, class_folder), os.path.join(train_data_dir, c, class_folder)) for class_folder in sample_dir[:int(split_percent * number_sample)]]
    
    [shutil.copy(os.path.join(dataset_path, c, class_folder), os.path.join(validation_data_dir, c, class_folder)) for class_folder in sample_dir[int(split_percent * number_sample):]]
    
    move_data(validation_data_dir)
    print(c + ' class folder splited!')

ClassX has 350 samples.
X class folder splited!
ClassFever has 350 samples.
Fever class folder splited!
ClassTwo has 352 samples.
Two class folder splited!
ClassOath has 350 samples.
Oath class folder splited!
ClassStand has 350 samples.
Stand class folder splited!
ClassU has 350 samples.
U class folder splited!
ClassSoldier has 350 samples.
Soldier class folder splited!
ClassSleep has 350 samples.
Sleep class folder splited!
ClassSunday has 350 samples.
Sunday class folder splited!
ClassA has 350 samples.
A class folder splited!
ClassEvening has 350 samples.
Evening class folder splited!
ClassCow has 352 samples.
Cow class folder splited!
ClassThumbs_up has 350 samples.
Thumbs_up class folder splited!
ClassElbow has 350 samples.
Elbow class folder splited!
ClassFive has 350 samples.
Five class folder splited!
ClassR has 350 samples.
R class folder splited!
ClassM has 350 samples.
M class folder splited!
ClassCough has 350 samples.
Cough class folder splited!
ClassClaw has 350 samples.

In [ ]:
# Load INCEPTIONV3
model=applications.InceptionV3(weights="imagenet", include_top=False, input_shape=(img_width, img_height, 3))
# Freeze first 15 layers
for layer in model.layers[:45]:
	layer.trainable = False
for layer in model.layers[45:]:
   layer.trainable = True
	
# Attach additional layers
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(95, activation="softmax")(x) # 4-way softmax classifier at the end

model_final = Model(inputs=model.input, outputs=predictions)

model_final.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(lr=1e-3, momentum=0.9), metrics=["accuracy"])

# train data generator (data augmentation)
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True,vertical_flip =True, fill_mode="nearest", zoom_range=0.3, width_shift_range=0.3, height_shift_range=0.3,channel_shift_range=0.3)
# test data generator (data augmentation)
test_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True,vertical_flip =True, fill_mode="nearest", zoom_range=0.3, width_shift_range=0.3, height_shift_range=0.3,channel_shift_range=0.3)

# load from directory
train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode="categorical")
# load from directory
validation_generator = test_datagen.flow_from_directory(validation_data_dir, target_size=(img_height, img_width), class_mode="categorical")

# save models

checkpoint = ModelCheckpoint("model_weights_1.hdf5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# early stopping
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

# TRAINING
history = model_final.fit_generator(train_generator, steps_per_epoch=1000, epochs=100, validation_data=validation_generator,validation_steps=500, callbacks=callbacks_list)

model_final.save_weights("inception_main_test_1.h5", overwrite = True)
model_json = model_final.to_json()


with open("model.json", "w") as json_file:
    json_file.write(model_json)


#training_set = train_datagen.flow_from_directory('validation',
 #                                                target_size = (300,300),
  #                                               batch_size = 8,
   #                                              class_mode = 'categorical')

#model.load_weights('inception_main.h5')

#X,y = training_set.next()
#result = model_final.predict_classes(X)
#print(result)

                  
#list all data in history
print(history.history.keys())
# summarize history for train
fig = plt.figure(figsize=(12,8))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
fig.savefig('model accuracy.jpg')
plt.close(fig)
#plt.show()

#plt.show()